In [1]:
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from itertools import combinations
import requests
import re
import numpy as np

## How to obtain information about the pathways of a given gene
First, the list of genes is loaded, as well as the index for the counts (the index of the counts is to filter out genes that I identified in papers but are not in our database)

In [2]:
genes = list(pd.read_csv('genes.csv')['name'])
index = pd.read_csv('pe-rna-counts.csv', index_col=0).index

Then a dictionary is created to convert the common name to the ensembl code, and the genes that we don't have in our database are filtered out to 71 genes

In [3]:
to_ensembl = pd.read_csv('genes.csv').set_index("name")["code"].to_dict()
genes = [g for g in genes if to_ensembl[g] in index]
len(genes)

71

So now we have our list of genes that we need to find pathways of

Then, a dictionary is created with keys of genes and values of pathways, using the PathwayCommons API. This takes around 17 seconds because APIs can take time 

In [4]:
d = {}
for g in tqdm(genes):
    d[g] = re.findall(r'"name":"(.*?)","dataSource":', requests.get("https://www.pathwaycommons.org/pc2/top_pathways?q=" + g).text)
d

  0%|          | 0/71 [00:00<?, ?it/s]

100%|██████████| 71/71 [00:16<00:00,  4.39it/s]


{'EEF1A1': [],
 'UBC': ['Ubiquitin–Proteasome Pathway',
  'NF-kB Signaling Pathway',
  'T Cell Receptor Signaling Pathway',
  'Circadian Clock',
  'Metabolism of proteins'],
 'FTL': ['Transport of small molecules'],
 'TMSB10': [],
 'HIST1H4A': ['Signaling events mediated by HDAC Class III',
  'Signaling events mediated by HDAC Class II',
  'Signaling events mediated by HDAC Class I'],
 'ZNF346': [],
 'BTG2': ['BTG Family Proteins and Cell Cycle Regulation',
  'Direct p53 effectors'],
 'INHBA': ['ALK1 signaling events',
  'Heterotrimeric GPCR signaling pathway (through G alpha i and pertussis toxin) ( GPCR signaling (pertussis toxin) )',
  'Heterotrimeric GPCR signaling pathway (through_G alpha s_ACs_PKA_BRaf_and_ERKcascade)(canonical) ( GPCR signaling (G alpha s, PKA and ERK) )',
  'TGF-beta signaling pathway',
  'Heterotrimeric GTP-binding protein coupled receptor signaling pathway (through G alpha i, adenylate cyclase and cAMP) ( GPCR signaling (G alpha i) )',
  'Heterotrimeric GPCR 

This is backwards from what we need though - what we need is a dictionary that maps pathways to genes, not genes to pathways, so we create the `pathways_to_genes` dict

In [5]:
pathways_to_genes = defaultdict(list)
for gene, pathways in d.items():
    for pathway in pathways:
        pathways_to_genes[pathway].append(gene)
path_to_genes = dict(pathways_to_genes)
path_to_genes

{'Ubiquitin–Proteasome Pathway': ['UBC'],
 'NF-kB Signaling Pathway': ['UBC', 'TNF'],
 'T Cell Receptor Signaling Pathway': ['UBC'],
 'Circadian Clock': ['UBC', 'BHLHE40', 'NRIP1', 'SERPINE1'],
 'Metabolism of proteins': ['UBC', 'ACE', 'APOE'],
 'Transport of small molecules': ['FTL'],
 'Signaling events mediated by HDAC Class III': ['HIST1H4A'],
 'Signaling events mediated by HDAC Class II': ['HIST1H4A'],
 'Signaling events mediated by HDAC Class I': ['HIST1H4A', 'TNF'],
 'BTG Family Proteins and Cell Cycle Regulation': ['BTG2'],
 'Direct p53 effectors': ['BTG2', 'NDRG1', 'SERPINE1'],
 'ALK1 signaling events': ['INHBA'],
 'Heterotrimeric GPCR signaling pathway (through G alpha i and pertussis toxin) ( GPCR signaling (pertussis toxin) )': ['INHBA',
  'IGF1',
  'LEP',
  'VEGFA',
  'VEGFB',
  'VEGFC',
  'TNF'],
 'Heterotrimeric GPCR signaling pathway (through_G alpha s_ACs_PKA_BRaf_and_ERKcascade)(canonical) ( GPCR signaling (G alpha s, PKA and ERK) )': ['INHBA',
  'IGF1',
  'LEP',
  'VE

Now we need to get all the pairs which share pathways and all the pairs that do not - which are the two groups, A and B, that you mentioned a few days ago

In [6]:
A = []
B = []
key_pairs = combinations(d.keys(), 2)
for key_pair in key_pairs:
    key1, key2 = key_pair
    if bool(set(d[key1]) & set(d[key2])):
        A.append((key1, key2))
    else:
        B.append((key1, key2))
print(len(A))
print(len(B))

140
2005
